<a href="https://colab.research.google.com/github/IT24102088/Plant-Disease-Detection-System-PG-NO-147/blob/main/Notebooks/combined_preprocessing_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Combined Preprocessing Pipeline

In [1]:
!pip install kaggle
import os
os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_KEY'] = ''



In [2]:
!kaggle datasets download -d "vipoooool/new-plant-diseases-dataset" -p /content/drive/MyDrive/datasets/

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
new-plant-diseases-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
from IPython.display import clear_output
!unzip /content/drive/MyDrive/datasets/new-plant-diseases-dataset.zip
clear_output()

In [67]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [68]:

training_set = tf.keras.utils.image_dataset_from_directory(
    '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/',
    image_size=(224, 224),
    batch_size=32,
    validation_split=None,
    subset=None,
)

training_set = tf.keras.utils.image_dataset_from_directory(
    '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/',
    image_size=(224, 224),
    batch_size=32,
    validation_split=None,
    subset=None,
)



Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.


In [69]:
from tensorflow.keras.layers import Rescaling
training_set = training_set.map(lambda x, y: (Rescaling(1./255)(x), y))

In [70]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

data_aug_geo = tf.keras.Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(0.15),
    RandomZoom(0.1)
])


In [71]:
from tensorflow.keras.layers import RandomBrightness, RandomContrast

data_aug_color = tf.keras.Sequential([
    RandomBrightness(factor=0.2),
    RandomContrast(0.2)
])

In [75]:
from tensorflow.keras.layers import GaussianNoise

noise_layer = GaussianNoise(0.1)


In [73]:
import cv2
import numpy as np

def hist_eq(image, label):
    image = cv2.cvtColor(image.numpy(), cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(image)
    l = cv2.equalizeHist(l)
    image = cv2.merge((l,a,b))
    image = cv2.cvtColor(image, cv2.COLOR_LAB2RGB)
    return tf.convert_to_tensor(image, dtype=tf.float32), label

training_set = training_set.map(lambda x, y: tf.py_function(hist_eq, [x, y], [tf.float32, tf.int32]))
